In [32]:
import sys
import pyccl as ccl
import pickle
import camb
sys.path.insert(0,'./')
# from importlib import reload

In [33]:
from distributed import LocalCluster
from dask.distributed import Client  # we already had this above
#http://distributed.readthedocs.io/en/latest/_modules/distributed/worker.html
c=LocalCluster(n_workers=1,processes=False,memory_limit='25gb',threads_per_worker=14,memory_spill_fraction=.99,
               memory_monitor_interval='2000ms')
client=Client(c)

02:45:29 DEBUG:Using selector: KqueueSelector
02:45:29 DEBUG:Using selector: KqueueSelector
/Users/Deep/anaconda/envs/py36/lib/python3.6/site-packages/distributed/dashboard/core.py:79: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


In [3]:
from cov_3X2 import *
from lsst_utils import *

In [4]:
#only for python3
import importlib
reload=importlib.reload

In [5]:
zs=lsst_source_tomo_bins()
corr_ll=('shear', 'shear')

ns0:  27.0


In [6]:
wigner_files={}
wigner_files[0]= '/Users/Deep/dask_temp/dask_wig3j_l3500_w2100_0_reorder.zarr'
wigner_files[2]= '/Users/Deep/dask_temp/dask_wig3j_l3500_w2100_2_reorder.zarr'



# Power Spectrum covariance

In [7]:
#setup parameters
lmax_cl=1000
lmin_cl=2
l0=np.arange(lmin_cl,lmax_cl)

lmin_cl_Bins=lmin_cl+10
lmax_cl_Bins=lmax_cl-10
Nl_bins=20
l_bins=np.int64(np.logspace(np.log10(lmin_cl_Bins),np.log10(lmax_cl_Bins),Nl_bins))
lb=np.sqrt(l_bins[1:]*l_bins[:-1])

l=np.unique(np.int64(np.logspace(np.log10(lmin_cl),np.log10(lmax_cl),Nl_bins*20))) #if we want to use fewer ell

do_cov=True
bin_cl=True

SSV_cov=True
tidal_SSV_cov=False
Tri_cov=True

bin_xi=True
theta_bins=np.logspace(np.log10(1./60),1,20)

##  Test with 1 redshift bin

In [8]:
store_win=True
window_lmax=2000

In [9]:
use_window=False
do_cov=False

In [10]:
z0=1 #1087
zs_bin1=source_tomo_bins(zp=[z0],p_zp=np.array([1]),ns=30,use_window=use_window,nside=1024)

In [11]:
use_binned_l=True

In [12]:
#use all ell
kappa0=cov_3X2(zs_bins=zs_bin1,do_cov=do_cov,bin_cl=bin_cl,l_bins=l_bins,l=l0, zg_bins=None,
                                       use_window=use_window,Tri_cov=Tri_cov,
                                       use_binned_l=use_binned_l,wigner_files=wigner_files,
                                       SSV_cov=SSV_cov,tidal_SSV_cov=tidal_SSV_cov,f_sky=0.35,
                                       store_win=store_win,window_lmax=window_lmax
                                       )

{('shear', 'shear'): [(0, 0)]} {'shear': 1, 'galaxy': 0, 'kappa': 0}
Win gen: step size 313
Window done
{('shear', 'shear'): [(0, 0)]} {'shear': 1, 'galaxy': 0, 'kappa': 0}
Win gen: step size 20
Window done


./cov_utils.py:60: RuntimeWarning: invalid value encountered in true_divide
  Win0=2*jn(1,l_th)/l_th


cl dict done dict_keys([('shear', 'shear')])
cl dict done dict_keys([('shear', 'shear')])
{('shear', 'shear'): [(0, 0)]} {'shear': 1, 'galaxy': 0, 'kappa': 0}
Win gen: step size 313
cl dict done dict_keys([('shear', 'shear')])
cl dict done dict_keys([('shear', 'shear')])
Window done


In [13]:
%time cl0G=kappa0.cl_tomo() 

cl dict done dict_keys([('shear', 'shear')])
CPU times: user 19.2 s, sys: 488 ms, total: 19.7 s
Wall time: 13.3 s


In [14]:
kappa0.l.shape

(19,)

In [15]:
cl0G['cl'].keys()

dict_keys([('shear', 'shear')])

In [16]:
%time cl0=cl0G['stack'].compute()

CPU times: user 122 ms, sys: 3.37 ms, total: 125 ms
Wall time: 16.6 ms


In [17]:
do_xi=True
bin_xi=True
do_cov=False
bin_cl=True
th_min=2.5/60
th_max=250./60
n_th_bins=20
th_bins=np.logspace(np.log10(th_min),np.log10(th_max),n_th_bins+1)
th=np.logspace(np.log10(th_min),np.log10(th_max),n_th_bins*40)
thb=np.sqrt(th_bins[1:]*th_bins[:-1])

In [18]:
# import wigner_transform 
# reload(wigner_transform)
# from wigner_transform import *

import binning
reload(binning)
from binning import *

In [19]:
#Hankel Transform setup
HT_kwargs={'l':l0,'theta':th,'m1_m2':[(2,2),(2,-2)]}
WT=wigner_transform(**HT_kwargs)

In [20]:
import cov_3X2
reload(cov_3X2)
from cov_3X2 import *

In [21]:
use_binned_theta=True

In [22]:
kappa_xi0=cov_3X2(zs_bins=zs_bin1,do_cov=do_cov,bin_cl=bin_cl,l_bins=l_bins,l=l0, zg_bins=None,
                                       use_window=use_window,Tri_cov=Tri_cov,
                                       wigner_files=wigner_files,
                                       SSV_cov=SSV_cov,tidal_SSV_cov=tidal_SSV_cov,f_sky=0.35,
                                       store_win=store_win,window_lmax=window_lmax,
                                      do_xi=do_xi,bin_xi=bin_xi,theta_bins=th_bins,HT=WT,
                                      use_binned_l=False,use_binned_theta=False
                                       )

{('shear', 'shear'): [(0, 0)]} {'shear': 1, 'galaxy': 0, 'kappa': 0}
Win gen: step size 313
Window done


In [23]:
kappa_xib=cov_3X2(zs_bins=zs_bin1,do_cov=do_cov,bin_cl=bin_cl,l_bins=l_bins,l=l0, zg_bins=None,
                                       use_window=use_window,Tri_cov=Tri_cov,
                                       wigner_files=wigner_files,
                                       SSV_cov=SSV_cov,tidal_SSV_cov=tidal_SSV_cov,f_sky=0.35,
                                       store_win=store_win,window_lmax=window_lmax,
                                      do_xi=do_xi,bin_xi=bin_xi,theta_bins=th_bins,HT=WT,
                                      use_binned_l=use_binned_l,use_binned_theta=use_binned_theta
                                       )

{('shear', 'shear'): [(0, 0)]} {'shear': 1, 'galaxy': 0, 'kappa': 0}
Win gen: step size 313
Window done
{('shear', 'shear'): [(0, 0)]} {'shear': 1, 'galaxy': 0, 'kappa': 0}
Win gen: step size 20
Window done
{('shear', 'shear'): [(0, 0)]} {'shear': 1, 'galaxy': 0, 'kappa': 0}
Win gen: step size 313
Window done
cl dict done dict_keys([('shear', 'shear')])
cl dict done dict_keys([('shear', 'shear')])
cl dict done dict_keys([('shear', 'shear')])
cl dict done dict_keys([('shear', 'shear')])
{('shear', 'shear'): [(0, 0)]} {'shear': 1, 'galaxy': 0, 'kappa': 0}
Win gen: step size 313
cl dict done dict_keys([('shear', 'shear')])
cl dict done dict_keys([('shear', 'shear')])
Window done


In [24]:
%time xiHG_b=kappa_xib.xi_tomo()

cl dict done dict_keys([('shear', 'shear')])
CPU times: user 16.9 s, sys: 2.62 s, total: 19.5 s
Wall time: 13 s


In [27]:
%time xiH_b=xiHG_b['stack'].compute()

CPU times: user 6.71 ms, sys: 1.3 ms, total: 8.01 ms
Wall time: 15.3 ms


In [29]:
%time xiHG0=kappa_xi0.xi_tomo()
%time xiH0=xiHG0['stack'].compute()

cl dict done dict_keys([('shear', 'shear')])
CPU times: user 17.8 s, sys: 1.7 s, total: 19.5 s
Wall time: 12.9 s
CPU times: user 176 ms, sys: 162 ms, total: 338 ms
Wall time: 23.4 ms


In [30]:
xiH0['xi']

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.55128358e-06, 1.08317907e-06, 6.46518955e-07,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 3.00315782e-06, 2.63244521e-06, 2.16202467e-06])

In [31]:
xiH_b['xi']

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.14106370e-06, 6.59476871e-07, 2.27708561e-07,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 3.02490036e-06, 2.63685408e-06, 2.16015253e-06])